# Coffee Quality Prediction ☕️☕️☕️

🗂️ Dataset: [Link](https://www.kaggle.com/datasets/fatihb/coffee-quality-data-cqi/data)

## Contents
- [Introduction](#Introduction)
- [Understanding Dataset](#Understanding-Dataset)
- [Data Quality](#Data-Quality)
    - [Filling Missing Values](##Filling-Missing-Values)
    - [Data Typo](##Data-Typo)
- [Exploring Data](#Exploring-Data)
    - [Numerical Data](##Numerical-Data)
    - [Categorical Data](##Categorical-Data)
    - [Remove Irrevant Data](##Remove-Irrevant-Data)


# Introduction

The dataset used in this project is the Coffee Quality Data from the Coffee Quality Institute (CQI). It contains detailed information about coffee metadata. I separated dataset into 4 main groups based on [Link](https://github.com/jldbc/coffee-quality-database)

`📊 Quality Measures`
- Aroma
- Flavor
- Aftertaste
- Acidity
- Body
- Balance
- Uniformity
- Clean Cup
- Sweetness
- Overall
- Total Cup Points - Calculated from 10 features above. 
- Defects
- Moisture Percentage
- Category One Defects
- Quakers
- Category Two Defects

`🌱 Bean Metadata`
- Harvest Year - has 2 formats 'YYYY-YYYY' and 'YYYY'
- Variety
- Processing Method
- Color


`👨🏻‍🌾 Farm and export Metadata`
- Country of Origin
- Farm Name
- Lot Number
- Mill - In this dataset, mill likely to refer to location name that process coffee more than type of mill. 
- ICO Number - used as a traceability code of coffee product from origin to customer [Link](http://www.ico.org/documents/icc-102-9e-rules-certificates-final.pdf)
- Company
- Altitude
- Region
- Producer
- Number of Bags
- Bag Weight
- In-Country Partner
- Owner

`🏅 Certification Metadata`
- Grading Date
- Expiration - In this dataset, I found that the period between expriation and grading date is 12 months. so I assume this data is exporation of grading score.
- Certification Body
- Certification Address
- Certification Contact
- Status

In this project, I want to create a prediction model for coffee quality 'Total Cup Points' which I will explore and analysis there is any features (except 10 features) have significant to be used in prediction or not. Note: we're already know exact function to compute 'Total Cup Points' using 10 quality features.

# Understanding-Dataset

In [1]:
# import libraries
import pandas as pd 
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import re
from plotly.subplots import make_subplots
import json
pd.set_option('display.max_columns', 100)

Here I explore about dataset overview.

In [2]:
filepath = 'data/df_arabica_clean.csv'
raw_df = pd.read_csv(filepath)
print(f"Original data shape: {raw_df.shape} | rows: {raw_df.shape[0]} | columns: {raw_df.shape[1]}")

Original data shape: (207, 41) | rows: 207 | columns: 41


In [3]:
# example of the data
raw_df.head()

,Unnamed: 0,ID,Country of Origin,Farm Name,Lot Number,Mill,ICO Number,Company,Altitude,Region,Producer,Number of Bags,Bag Weight,In-Country Partner,Harvest Year,Grading Date,Owner,Variety,Status,Processing Method,Aroma,Flavor,Aftertaste,Acidity,Body,Balance,Uniformity,Clean Cup,Sweetness,Overall,Defects,Total Cup Points,Moisture Percentage,Category One Defects,Quakers,Color,Category Two Defects,Expiration,Certification Body,Certification Address,Certification Contact
0,0,0,Colombia,Finca El Paraiso,CQU2022015,Finca El Paraiso,NaN,Coffee Quality Union,1700-1930,"Piendamo,Cauca",Diego Samuel Bermudez,1,35 kg,Japan Coffee Exchange,2021 / 2022,"September 21st, 2022",Coffee Quality Union,Castillo,Completed,Double Anaerobic Washed,8.58,8.50,8.42,8.58,8.25,8.42,10.0,10.0,10.0,8.58,0.0,89.33,11.8,0,0,green,3,"September 21st, 2023",Japan Coffee Exchange,"〒413-0002 静岡県熱海市伊豆山１１７３−５８ 1173-58 Izusan, Ata...",松澤 宏樹 Koju Matsuzawa - +81(0)9085642901
1,1,1,Taiwan,Royal Bean Geisha Estate,"The 2022 Pacific Rim Coffee Summit,T037",Royal Bean Geisha Estate,NaN,Taiwan Coffee Laboratory,1200,Chiayi,曾福森,1,80 kg,Taiwan Coffee Laboratory 台灣咖啡研究室,2021 / 2022,"November 15th, 2022",Taiwan Coffee Laboratory 台灣咖啡研究室,Gesha,Completed,Washed / Wet,8.50,8.50,7.92,8.00,7.92,8.25,10.0,10.0,10.0,8.50,0.0,87.58,10.5,0,0,blue-green,0,"November 15th, 2023",Taiwan Coffee Laboratory 台灣咖啡研究室,"QAHWAH CO., LTD 4F, No. 225, Sec. 3, Beixin Rd...","Lin, Jen-An Neil 林仁安 - 886-289116612"
2,2,2,Laos,OKLAO coffee farms,"The 2022 Pacific Rim Coffee Summit,LA01",oklao coffee processing plant,NaN,Taiwan Coffee Laboratory,1300,Laos Borofen Plateau,WU TAO CHI,19,25 kg,Taiwan Coffee Laboratory 台灣咖啡研究室,2021 / 2022,"November 15th, 2022",Taiwan Coffee Laboratory 台灣咖啡研究室,Java,Completed,Semi Washed,8.33,8.42,8.08,8.17,7.92,8.17,10.0,10.0,10.0,8.33,0.0,87.42,10.4,0,0,yellowish,2,"November 15th, 2023",Taiwan Coffee Laboratory 台灣咖啡研究室,"QAHWAH CO., LTD 4F, No. 225, Sec. 3, Beixin Rd...","Lin, Jen-An Neil 林仁安 - 886-289116612"
3,3,3,Costa Rica,La Cumbre,CQU2022017,La Montana Tarrazu MIll,NaN,Coffee Quality Union,1900,"Los Santos,Tarrazu",Santa Maria de Dota,1,22 kg,Japan Coffee Exchange,2022,"September 21st, 2022",Coffee Quality Union,Gesha,Completed,Washed / Wet,8.08,8.17,8.17,8.25,8.17,8.08,10.0,10.0,10.0,8.25,0.0,87.17,11.8,0,0,green,0,"September 21st, 2023",Japan Coffee Exchange,"〒413-0002 静岡県熱海市伊豆山１１７３−５８ 1173-58 Izusan, Ata...",松澤 宏樹 Koju Matsuzawa - +81(0)9085642901
4,4,4,Colombia,Finca Santuario,CQU2023002,Finca Santuario,NaN,Coffee Quality Union,1850-2100,"Popayan,Cauca",Camilo Merizalde,2,24 kg,Japan Coffee Exchange,2022,"March 6th, 2023",Coffee Quality Union,Red Bourbon,Completed,"Honey,Mossto",8.33,8.33,8.08,8.25,7.92,7.92,10.0,10.0,10.0,8.25,0.0,87.08,11.6,0,2,yellow-green,2,"March 5th, 2024",Japan Coffee Exchange,"〒413-0002 静岡県熱海市伊豆山１１７３−５８ 1173-58 Izusan, Ata...",松澤 宏樹 Koju Matsuzawa - +81(0)9085642901


In [4]:
# rename columns to make them more readable
columns = list(raw_df.columns)
rename_columns = {column:re.sub(r'\s+', '_', column.lower()) for column in columns}
# In-Country Partner column has a special character, I will replace it with '_'
rename_columns['In-Country Partner'] = 'in_country_partner'
# rename the columns
raw_df.rename(columns=rename_columns, inplace=True)
raw_df.head()

,unnamed:_0,id,country_of_origin,farm_name,lot_number,mill,ico_number,company,altitude,region,producer,number_of_bags,bag_weight,in_country_partner,harvest_year,grading_date,owner,variety,status,processing_method,aroma,flavor,aftertaste,acidity,body,balance,uniformity,clean_cup,sweetness,overall,defects,total_cup_points,moisture_percentage,category_one_defects,quakers,color,category_two_defects,expiration,certification_body,certification_address,certification_contact
0,0,0,Colombia,Finca El Paraiso,CQU2022015,Finca El Paraiso,NaN,Coffee Quality Union,1700-1930,"Piendamo,Cauca",Diego Samuel Bermudez,1,35 kg,Japan Coffee Exchange,2021 / 2022,"September 21st, 2022",Coffee Quality Union,Castillo,Completed,Double Anaerobic Washed,8.58,8.50,8.42,8.58,8.25,8.42,10.0,10.0,10.0,8.58,0.0,89.33,11.8,0,0,green,3,"September 21st, 2023",Japan Coffee Exchange,"〒413-0002 静岡県熱海市伊豆山１１７３−５８ 1173-58 Izusan, Ata...",松澤 宏樹 Koju Matsuzawa - +81(0)9085642901
1,1,1,Taiwan,Royal Bean Geisha Estate,"The 2022 Pacific Rim Coffee Summit,T037",Royal Bean Geisha Estate,NaN,Taiwan Coffee Laboratory,1200,Chiayi,曾福森,1,80 kg,Taiwan Coffee Laboratory 台灣咖啡研究室,2021 / 2022,"November 15th, 2022",Taiwan Coffee Laboratory 台灣咖啡研究室,Gesha,Completed,Washed / Wet,8.50,8.50,7.92,8.00,7.92,8.25,10.0,10.0,10.0,8.50,0.0,87.58,10.5,0,0,blue-green,0,"November 15th, 2023",Taiwan Coffee Laboratory 台灣咖啡研究室,"QAHWAH CO., LTD 4F, No. 225, Sec. 3, Beixin Rd...","Lin, Jen-An Neil 林仁安 - 886-289116612"
2,2,2,Laos,OKLAO coffee farms,"The 2022 Pacific Rim Coffee Summit,LA01",oklao coffee processing plant,NaN,Taiwan Coffee Laboratory,1300,Laos Borofen Plateau,WU TAO CHI,19,25 kg,Taiwan Coffee Laboratory 台灣咖啡研究室,2021 / 2022,"November 15th, 2022",Taiwan Coffee Laboratory 台灣咖啡研究室,Java,Completed,Semi Washed,8.33,8.42,8.08,8.17,7.92,8.17,10.0,10.0,10.0,8.33,0.0,87.42,10.4,0,0,yellowish,2,"November 15th, 2023",Taiwan Coffee Laboratory 台灣咖啡研究室,"QAHWAH CO., LTD 4F, No. 225, Sec. 3, Beixin Rd...","Lin, Jen-An Neil 林仁安 - 886-289116612"
3,3,3,Costa Rica,La Cumbre,CQU2022017,La Montana Tarrazu MIll,NaN,Coffee Quality Union,1900,"Los Santos,Tarrazu",Santa Maria de Dota,1,22 kg,Japan Coffee Exchange,2022,"September 21st, 2022",Coffee Quality Union,Gesha,Completed,Washed / Wet,8.08,8.17,8.17,8.25,8.17,8.08,10.0,10.0,10.0,8.25,0.0,87.17,11.8,0,0,green,0,"September 21st, 2023",Japan Coffee Exchange,"〒413-0002 静岡県熱海市伊豆山１１７３−５８ 1173-58 Izusan, Ata...",松澤 宏樹 Koju Matsuzawa - +81(0)9085642901
4,4,4,Colombia,Finca Santuario,CQU2023002,Finca Santuario,NaN,Coffee Quality Union,1850-2100,"Popayan,Cauca",Camilo Merizalde,2,24 kg,Japan Coffee Exchange,2022,"March 6th, 2023",Coffee Quality Union,Red Bourbon,Completed,"Honey,Mossto",8.33,8.33,8.08,8.25,7.92,7.92,10.0,10.0,10.0,8.25,0.0,87.08,11.6,0,2,yellow-green,2,"March 5th, 2024",Japan Coffee Exchange,"〒413-0002 静岡県熱海市伊豆山１１７３−５８ 1173-58 Izusan, Ata...",松澤 宏樹 Koju Matsuzawa - +81(0)9085642901


In [5]:
# dataframe to overview information function
def get_info(df):
    """
    This function returns the information of the dataframe
    I separate the data into two dataframes, one for numeric data and the other for object data
    Args:
        df: pd.DataFrame
    Returns:
        numeric_df: pd.DataFrame
        object_df: pd.DataFrame
    """
    numeric_data = []
    object_data = []
    for column in df.columns:
        total = df[column].count() + df[column].isnull().sum()
        if df[column].dtype == object or df[column].dtype == bool:
            object_data.append({
                'column': column,
                'dtype': df[column].dtype,
                'valid': df[column].count(),
                'missing': df[column].isnull().sum(),
                'unique': df[column].nunique(),
                'unique_pct': round((df[column].nunique() *100 / total), 2),
                'top': df[column].mode().values[0],
                'freq': df[column].value_counts().values[0]
            })
        else:
            numeric_data.append({
                'column': column,
                'dtype': df[column].dtype,
                'valid': df[column].count(),
                'missing': df[column].isnull().sum(),
                'unique': df[column].nunique(),
                'unique_pct': round((df[column].nunique()*100 / total), 2),
                'mean': round(df[column].mean(), 2),
                'std': round(df[column].std(), 2),
                'min': df[column].min(),
                '25%': df[column].quantile(0.25),
                '50%': df[column].quantile(0.50),
                '75%': df[column].quantile(0.75),
                'max': df[column].max(),
            })
    numeric_df = pd.DataFrame(numeric_data)
    object_df = pd.DataFrame(object_data)
    return numeric_df, object_df


In [6]:
numeric_df, object_df = get_info(raw_df)

In [7]:
print("Numerical data:\n")
numeric_df

Numerical data:



,column,dtype,valid,missing,unique,unique_pct,mean,std,min,25%,50%,75%,max
0,unnamed:_0,int64,207,0,207,100.00,103.00,59.90,0.00,51.50,103.00,154.500,206.00
1,id,int64,207,0,207,100.00,103.00,59.90,0.00,51.50,103.00,154.500,206.00
2,number_of_bags,int64,207,0,55,26.57,155.45,244.48,1.00,1.00,14.00,275.000,2240.00
3,aroma,float64,207,0,19,9.18,7.72,0.29,6.50,7.58,7.67,7.920,8.58
4,flavor,float64,207,0,19,9.18,7.74,0.28,6.75,7.58,7.75,7.920,8.50
5,aftertaste,float64,207,0,20,9.66,7.60,0.28,6.67,7.42,7.58,7.750,8.42
6,acidity,float64,207,0,19,9.18,7.69,0.26,6.83,7.50,7.67,7.875,8.58
7,body,float64,207,0,17,8.21,7.64,0.23,6.83,7.50,7.67,7.750,8.25
8,balance,float64,207,0,18,8.70,7.64,0.26,6.67,7.50,7.67,7.790,8.42
9,uniformity,float64,207,0,3,1.45,9.99,0.10,8.67,10.00,10.00,10.000,10.00


In [8]:
print("Object data:\n")
object_df

Object data:



,column,dtype,valid,missing,unique,unique_pct,top,freq
0,country_of_origin,object,207,0,22,10.63,Taiwan,61
1,farm_name,object,205,2,172,83.09,Doi Tung Development Project,7
2,lot_number,object,206,1,187,90.34,1,11
3,mill,object,204,3,162,78.26,Dry Mill,11
4,ico_number,object,75,132,67,32.37,non,5
5,company,object,207,0,72,34.78,Taiwan Coffee Laboratory,51
6,altitude,object,206,1,97,46.86,1200,23
7,region,object,205,2,120,57.97,Chiayi,12
8,producer,object,206,1,172,83.09,Doi Tung Development Project,7
9,bag_weight,object,207,0,39,18.84,30 kg,39


In [9]:
# From `numeric_df` found that ['unnamed:_0', 'id'] are just index columns so I will discard these.
df = raw_df.drop(['unnamed:_0', 'id'], axis=1)
print(f"Data shape after dropping ['unnamed:_0', 'id']: {df.shape} | rows: {df.shape[0]} | columns: {df.shape[1]}")

Data shape after dropping ['unnamed:_0', 'id']: (207, 39) | rows: 207 | columns: 39


# Data-Quality

## Filling-Missing-Values

In [10]:
# Since numerical data has no missing values, I will focus on object data
object_df.query("missing > 0").sort_values(by='missing', ascending=False)

,column,dtype,valid,missing,unique,unique_pct,top,freq
4,ico_number,object,75,132,67,32.37,non,5
14,variety,object,201,6,48,23.19,Caturra,27
16,processing_method,object,202,5,10,4.83,Washed / Wet,124
3,mill,object,204,3,162,78.26,Dry Mill,11
1,farm_name,object,205,2,172,83.09,Doi Tung Development Project,7
7,region,object,205,2,120,57.97,Chiayi,12
2,lot_number,object,206,1,187,90.34,1,11
6,altitude,object,206,1,97,46.86,1200,23
8,producer,object,206,1,172,83.09,Doi Tung Development Project,7


### 1. ICO Number

To get 'ico_number', the coffee must meet their export quality standards (P16 in the [link](http://www.ico.org/documents/icc-102-9e-rules-certificates-final.pdf)). Instead of discard this column, I decide to transform it to boolean datatype.

In [11]:
# From tabe above, top of 'ico_number' is 'non' which is not a valid value. I will replace it with np.nan
df['ico_number'] = df['ico_number'].replace('non', np.nan)

# Convert 'ico_number' to boolean
df['ico_number'] = df['ico_number'].notnull()
df['ico_number'].value_counts(normalize=True)

ico_number
False    0.661836
True     0.338164
Name: proportion, dtype: float64

### 2. Altitude

I decide to consider 'altitude' before variety in case of 'altitude' will be considered to fill missing variety.

In [12]:
print("Unique Values:", df['altitude'].unique())

def clean_altitude(x):
    """
    This function cleans the 'altitude' column
    Args:
        x: str
    Returns:
        int
    """
    if pd.isnull(x):
        return x
    match = re.match("(\d+)-(\d+)", x)
    if match:
        return (int(match.group(1)) + int(match.group(2))) / 2
    else:
        return int(x)

# Clean data  '~', ' - ', ' A ' to '-'
df['altitude'] = df['altitude'].str.replace('~', '-').str.replace(' - ', '-').str.replace(' A ', '-')
# Convert 'altitude' to numeric if there is range, I will take the average
df['altitude'] = df['altitude'].apply(lambda x: clean_altitude(x))
print("Cleaned Unique Values:", df['altitude'].unique())

Unique Values: ['1700-1930' '1200' '1300' '1900' '1850-2100' '1668' '1250' '1400-1700'
 '1800-2200' '2000' '1900-2000' '1850' '1100' '1900-2100' '1570-1600'
 '850' '1500-1700' '1350' '1060' '600' '150-250' '668' '1950'
 '2150 - 2350' '1700' '650' '1600-1900' '300-500' '1000' '800' '1905'
 '150' '1600' '4700' '350-400' '230' '1750' '1654' '1400' '4895' '200-300'
 '700' '1450' '520' '1368' '1943' '400' '1300-1400' '2361' '1500' '2100'
 '1250-1350' '1500-1600' '1800' '1600-1750' '1411' '4895 A 5650' '400-600'
 nan '200-400' '1470' '250-400' '250-300' '1200 - 1580' '1400 - 1900'
 '1280-1325' '300' '750' '1300-1500' '950' '1100-1200' '1390' '340'
 '1200 - 1300' '1650' '1280' '900' '165' '465' '640' '1340' '139'
 '1200-1350' '1040' '140' '1500-1950' '460' '500' '800-1200' '1350-1550'
 '1200~1600' '5400' '900-1000' '1574' '435' '600-800' '1300-1800'
 '850-1100']
Cleaned Unique Values: [1815.  1200.  1300.  1900.  1975.  1668.  1250.  1550.  2000.  1950.
 1850.  1100.  1585.   850.  1600.  135

In [13]:
# plot relationship between 'total_cup_points' 'country_of_origin' and 'altitude'
fig = px.scatter(df, x='country_of_origin', y='altitude', color='total_cup_points', title='Total cup points vs Country of Origin vs Altitude')
fig.update_layout(width=800, height=800, xaxis={'categoryorder':'total ascending'})
fig.show()

In [14]:
# boxplot of 'country_of_origin' vs 'altitude'
fig = px.box(df, x='country_of_origin', y='altitude', title='Country of Origin vs Altitude')
fig.update_layout(width=800, height=800, xaxis={'categoryorder':'total ascending'})
fig.show()

In [15]:
df[df['altitude'].isnull()]

,country_of_origin,farm_name,lot_number,mill,ico_number,company,altitude,region,producer,number_of_bags,bag_weight,in_country_partner,harvest_year,grading_date,owner,variety,status,processing_method,aroma,flavor,aftertaste,acidity,body,balance,uniformity,clean_cup,sweetness,overall,defects,total_cup_points,moisture_percentage,category_one_defects,quakers,color,category_two_defects,expiration,certification_body,certification_address,certification_contact
105,Colombia,NaN,NaN,NaN,False,Coffee Quality Institute,NaN,NaN,NaN,1,1 kg,Barista and Coffee Academy of Asia,2022,"September 26th, 2022",Coffee Quality Institute,NaN,Completed,NaN,7.83,7.75,7.5,7.58,7.67,7.67,10.0,10.0,10.0,7.67,0.0,83.67,12.4,1,0,greenish,9,"September 26th, 2023",Barista and Coffee Academy of Asia,"The Place Bldg., Tunasan, Muntinlupa City, Met...",Rosario Cruz - +63 995 344 5688


In [16]:
# Fill missing values in 'altitude' with the median of the country
df['altitude'] = df.groupby('country_of_origin')['altitude'].transform(lambda x: x.fillna(x.median()))
df.loc[105, 'altitude']

1625.0

### 3. Variety

In this value, I decide to filling missing value of 'variety' with 'country_of_origin' and 'altitude'.
If missing 'variety' has no match 'altitude', the median of 'variety' in the same country is filled.

In [17]:
print("Unique Values:", df['variety'].unique())

# From unique values there are some values represent nan values ['unknown', 'unknow'], I will replace them with np.nan
df['variety'] = df['variety'].replace(['unknown', 'unknow'], np.nan)

print("\nCleaned Unique Values:", df['variety'].unique())

Unique Values: ['Castillo' 'Gesha' 'Java' 'Red Bourbon' 'Sl34+Gesha' 'SL34' 'Bourbon'
 'Ethiopian Heirlooms' 'Caturra' 'Wolishalo,Kurume,Dega' 'Typica'
 'Catimor' 'Castillo Paraguaycito' nan 'SL28' 'SL14' 'Catuai'
 'Yellow Bourbon' 'Catrenic' 'unknown' 'Pacamara'
 'Castillo and Colombia blend' 'Jember,TIM-TIM,Ateng'
 'BOURBON, CATURRA Y CATIMOR' 'Bourbon Sidra' 'Sarchimor'
 'Catimor,Catuai,Caturra,Bourbon' 'Parainema' 'SHG' 'Typica + SL34'
 'MARSELLESA, CATUAI, CATURRA & MARSELLESA, ANACAFE 14, CATUAI'
 'Mundo Novo' 'Red Bourbon,Caturra' 'Lempira' 'Typica Gesha' 'Gayo'
 'Bourbon, Catimor, Caturra, Typica' 'unknow' 'Maragogype'
 'Caturra-Catuai' 'SL28,SL34,Ruiru11' 'Yellow Catuai' 'Catucai'
 'Santander' 'Typica Bourbon Caturra Catimor' 'Caturra,Colombia,Castillo'
 'Castillo,Caturra,Bourbon' 'Pacas' 'Catuai and Mundo Novo']

Cleaned Unique Values: ['Castillo' 'Gesha' 'Java' 'Red Bourbon' 'Sl34+Gesha' 'SL34' 'Bourbon'
 'Ethiopian Heirlooms' 'Caturra' 'Wolishalo,Kurume,Dega' 'Typica'
 'Cat

In [18]:
non_variety_df = df[df['variety'].isnull()]
# filter the variety with country_of_origin and altitude to fill the missing values
non_variety_df[['country_of_origin', 'altitude']].value_counts()

country_of_origin  altitude
Brazil             1200.0      1
Taiwan             300.0       1
                   600.0       1
                   520.0       1
                   500.0       1
                   460.0       1
                   435.0       1
                   340.0       1
                   325.0       1
                   165.0       1
Brazil             1250.0      1
Taiwan             150.0       1
                   140.0       1
                   139.0       1
Indonesia          1350.0      1
Guatemala          1450.0      1
Colombia           1625.0      1
                   1411.0      1
Taiwan             1150.0      1
Name: count, dtype: int64

In [19]:
non_variety_df['country_of_origin'].unique()

array(['Brazil', 'Taiwan', 'Colombia', 'Guatemala', 'Indonesia'],
      dtype=object)

In [20]:
# plot relationship between 'variety' 'country_of_origin' and 'altitude'
# filter df 'country_of_origin' when has missing values
country_list = non_variety_df['country_of_origin'].unique()
variety_df = df[df['country_of_origin'].isin(country_list)]
fig = px.scatter(variety_df, x='variety', y='altitude', color='country_of_origin', title='Variety vs Country of Origin vs Altitude')
fig.update_layout(width=800, height=800, xaxis={'categoryorder':'total ascending'}, margin=go.layout.Margin(l=50))
fig.show()

In [21]:
# fill missing values in 'variety' with the altitude and country_of_origin if altitude is not equal find the nearest altitude
for index, row in non_variety_df.iterrows():
    country = row['country_of_origin']
    altitude = row['altitude']
    if pd.isnull(altitude):
        continue
    variety = df[(df['country_of_origin'] == country) & (df['altitude'] == altitude)]['variety'].mode().values
    if len(variety) > 0:
        df.loc[index, 'variety'] = variety[0]
    else:
        variety = df[(df['country_of_origin'] == country)]['variety'].mode().values
        if len(variety) > 0:
            df.loc[index, 'variety'] = variety[0]

In [22]:
non_variety_df = df[df['variety'].isnull()]
non_variety_df[['country_of_origin', 'altitude']].value_counts()

Series([], Name: count, dtype: int64)

In [23]:
# plot relationship between 'variety' 'country_of_origin' and 'altitude'
variety_df = df[df['country_of_origin'].isin(country_list)]
fig = px.scatter(variety_df, x='variety', y='altitude', color='country_of_origin', title='After Fill Missing Value Variety vs Country of Origin vs Altitude')
fig.update_layout(width=800, height=800, xaxis={'categoryorder':'total ascending'}, margin=go.layout.Margin(l=50))
fig.show()

### 4. Processing Method

In this value, I decide to filling missing value of 'processing_method' with 'country_of_origin' and 'variety' since 'altitude' information can be represented by 'variety'.

In [24]:
print("Unique Values:", df['processing_method'].unique())

Unique Values: ['Double Anaerobic Washed' 'Washed / Wet' 'Semi Washed' 'Honey,Mossto'
 'Natural / Dry' 'Pulped natural / honey' nan
 'Double Carbonic Maceration / Natural' 'Wet Hulling' 'Anaerobico 1000h'
 'SEMI-LAVADO']


In [25]:
# plot relationship between 'processing_method' 'country_of_origin' and 'variety'
fig = px.scatter(df, x='variety', y='processing_method', color='country_of_origin', title='Processing Method vs Country of Origin vs Variety')
fig.update_layout(width=800, height=800, xaxis={'categoryorder':'total ascending'}, margin=go.layout.Margin(l=50))
fig.show()

In [26]:
# fill missing values in 'processing_method' with the condition of country_of_origin and variety
non_processing_method_df = df[df['processing_method'].isnull()]
non_processing_method_df[['country_of_origin', 'variety']].value_counts()

country_of_origin  variety              
Colombia           Castillo Paraguaycito    1
                   Caturra                  1
Indonesia          Catimor                  1
Taiwan             Gesha                    1
                   Typica                   1
Name: count, dtype: int64

In [27]:
for index, row in non_processing_method_df.iterrows():
    country = row['country_of_origin']
    variety = row['variety']
    if pd.isnull(variety):
        continue
    processing_method = df[(df['country_of_origin'] == country) & (df['variety'] == variety)]['processing_method'].mode().values
    if len(processing_method) > 0:
        df.loc[index, 'processing_method'] = processing_method[0]
    else:
        variety = df[(df['country_of_origin'] == country)]['processing_method'].mode().values
        if len(variety) > 0:
            df.loc[index, 'processing_method'] = variety[0]

In [28]:
df[df['processing_method'].isnull()]

,country_of_origin,farm_name,lot_number,mill,ico_number,company,altitude,region,producer,number_of_bags,bag_weight,in_country_partner,harvest_year,grading_date,owner,variety,status,processing_method,aroma,flavor,aftertaste,acidity,body,balance,uniformity,clean_cup,sweetness,overall,defects,total_cup_points,moisture_percentage,category_one_defects,quakers,color,category_two_defects,expiration,certification_body,certification_address,certification_contact


In [29]:
df.loc[178, :]

country_of_origin                                                Indonesia
farm_name                                              Wahana Graha Makmur
lot_number                                                    015/1686/057
mill                                         PT. Wahana Graha Makmur Medan
ico_number                                                           False
company                                               InterAmerican Coffee
altitude                                                            1350.0
region                                                             Sumatra
producer                                     PT. Wahana Graha Makmur Group
number_of_bags                                                         320
bag_weight                                                           60 kg
in_country_partner       NKG Quality Service (a division of Bernhard Ro...
harvest_year                                                          2022
grading_date             

### 5. Mill

From my search 'mill' refers to facilities where coffee is processed. There are 2 main types
- Wet mill
- Dry mill


mill is the location where processing methods are applied. In this dataset found that 'mill' is likely to refer to location name more than type of mill which I thought that is quite specific. Moreover most of 'mill' data is not in english which made hard to classify mill type so I decided to discard this data. 

In [30]:
print(df['mill'].unique())

df['mill'].value_counts(normalize=True) 

['Finca El Paraiso' 'Royal Bean Geisha Estate'
 'oklao coffee processing plant' 'La Montana Tarrazu MIll'
 'Finca Santuario' 'Dinámica Café' '野牡丹咖啡' '七彩琉璃咖啡莊園' '亮軒咖啡莊園'
 'GOURMET COFFEE MILL' 'Moledo社\u3000委託精選場' 'Cafetoland' '古峰咖啡莊園'
 'Dry Mill or Hulling Facility' 'La Gaitania' '青葉咖啡莊園'
 'Halo Bariti Cooprative' 'yes' 'El Vergel'
 '仲大叔咖啡莊園Uncle Chung.s Coffee Farm'
 'Machines to peel outer skin of the cherry coffee and coffee is dried manually'
 'El Diamante' '永舜咖啡莊園' 'Dry mill' '鄒築園ZouZhouYuan' '嵩岳咖啡莊園' '他扶芽有機農園'
 'Kona Coffee & Tea' '鄉庭有機農場Siang-Ting Organic Farm'
 '金讚咖啡農莊園(Jinzan Coffee Estate)' 'Lata Agri export' 'dry mill'
 'GUJI COFFEE EXPORT P.L.C' '卓武山咖啡農場' '御香咖啡園 YU SIANG Coffee Estate' 'NKG'
 '花蓮縣秀林鄉特用作物（咖啡）產銷第一班Agriculture Production and Marketing Groups of Hualien Shlin township special crop (coffee) 1st class'
 '乾燥機、脫殼機' '香夾蘭莊園' 'Dry Mill' '愛姬咖啡莊園iGfarm' '青山坪咖啡農場'
 'Beneficio Pastores' 'BENEFICIO LAS AMERICAS'
 'inmobiliaria e inversiones dos mil, s.a.' '六曲窩' '北平山林' 'PT 

mill
Dry Mill                 0.053922
yes                      0.024510
Dry mill                 0.019608
GOURMET COFFEE MILL      0.019608
Agua Caliente            0.014706
                           ...   
名陽園                      0.004902
CupnCare                 0.004902
苗55咖啡莊園                  0.004902
5.2Ha                    0.004902
Beneficio humedo/seco    0.004902
Name: proportion, Length: 162, dtype: float64

In [31]:
# plot bar chart for mill
fig = go.Figure()
# bar plot
fig.add_trace(go.Bar(x=df['mill'].value_counts().index, y=df['mill'].value_counts().values, marker_color='blue',))
fig.update_layout(title='Mill Distribution', xaxis_title='Mill', yaxis_title='Count')
fig.show()

In [32]:
from langdetect import detect_langs

def detect_language(text):
    try:
        return detect_langs(text)[0].lang
    except Exception as e:
        return 'unknown'

# detect language of the 'mill' column
df_mill = df[['mill']].copy()
df_mill['mill_lang'] = df_mill['mill'].apply(detect_language)
# if mill_lang != 'en' then replace with 'other'
df_mill.loc[df_mill['mill_lang'] != 'en', 'mill_lang'] = 'other'
print(df_mill['mill_lang'].value_counts(normalize=True))
print(f"\nData in 'mill' column that is not in English: {df_mill['mill_lang'].value_counts(normalize=True)['other']:.2%}")
print(f"Data in 'mill' column that is in English: {df_mill['mill_lang'].value_counts(normalize=True)['en']:.2%}")

mill_lang
other    0.748792
en       0.251208
Name: proportion, dtype: float64

Data in 'mill' column that is not in English: 74.88%
Data in 'mill' column that is in English: 25.12%


### 6. Farm Name

The same reason with 'mill' that 'farm_name' data consisting of specific name and most of them is not in english moreover their most frequent name contain only ~3% of total data so I decided to not consider this field to be one of feature.

In [33]:
print(df['farm_name'].unique())

df['farm_name'].value_counts(normalize=True)

['Finca El Paraiso' 'Royal Bean Geisha Estate' 'OKLAO coffee farms'
 'La Cumbre' 'Finca Santuario' 'La Colina'
 '野牡丹咖啡莊園 Melastoma Coffee Estate'
 '七彩琉璃咖啡莊園 Chi Tsai Liu Li Ecological Farm'
 '亮軒咖啡莊園 Liang Xuan Coffee Farm' 'MASHIMA AMCOS' 'TADE GG'
 'Karen Acajabon Coffee Farm' '古峰咖啡莊園 Goodfun Coffee Farm' 'Gelana Geisha'
 'La Gaitania' '青葉咖啡莊園' 'Halo Bariti Cooprative' '東璧咖啡莊園' '櫻桃果古坑咖啡莊園'
 'BURKA' 'Finca Vista Hermosa' '仲大叔咖啡莊園Uncle Chung.s Coffee Farm'
 'Hom Doi' 'El Diamante' '永舜咖啡莊園' 'Fazenda Recreio' '鄒築園ZouZhouYuan'
 '嵩岳咖啡莊園' '他扶芽有機農園' 'Hokukano Ranch' '鄉庭有機農場Siang-Ting Organic Farm'
 '金讚咖啡農莊園(Jinzan Coffee Estate)' 'Eshetu farm'
 'Uraga Bisrat washing station' 'YHAENU PLC FARM' '卓武山咖啡農場'
 '御香咖啡園 YU SIANG Coffee Estate' 'Small Holder' '皇庭咖啡莊園'
 '花蓮縣秀林鄉特用作物（咖啡）產銷第一班Agriculture Production and Marketing Groups of Hualien Shlin township special crop (coffee) 1st class'
 '八一高地' '香夾蘭莊園' 'Gideon Kule' '愛姬咖啡莊園iGfarm' '青山坪咖啡農場' 'varias fincas'
 'PEÑA BLANCA' 'san francisco cotzal' '六曲窩' 

farm_name
Doi Tung Development Project    0.034146
La Cuchilla                     0.024390
Gamboa Farm                     0.014634
Agua Caliente                   0.014634
MASHIMA AMCOS                   0.014634
                                  ...   
名陽園                             0.004878
CupnCare                        0.004878
苗55咖啡莊園                         0.004878
TRACON TRADING PLC              0.004878
Walter Matter                   0.004878
Name: proportion, Length: 172, dtype: float64

In [34]:
# plot bar chart for farm_name
fig = go.Figure()
# bar plot
fig.add_trace(go.Bar(x=df['farm_name'].value_counts().index, y=df['farm_name'].value_counts().values, marker_color='blue',))
fig.update_layout(title='Farm Name Distribution', xaxis_title='Farm Name', yaxis_title='Count')
fig.show()

In [35]:
# detect language of the 'farm_name' column
df_farm = df[['farm_name']].copy()
df_farm['farm_name_lang'] = df_farm['farm_name'].apply(detect_language)
# if farm_name_lang != 'en' then replace with 'other'
df_farm.loc[df_farm['farm_name_lang'] != 'en', 'farm_name_lang'] = 'other'
print(df_farm['farm_name_lang'].value_counts(normalize=True))
print(f"\nData in 'farm_name' column that is not in English: {df_farm['farm_name_lang'].value_counts(normalize=True)['other']:.2%}")
print(f"Data in 'farm_name' column that is in English: {df_farm['farm_name_lang'].value_counts(normalize=True)['en']:.2%}")

farm_name_lang
other    0.879227
en       0.120773
Name: proportion, dtype: float64

Data in 'farm_name' column that is not in English: 87.92%
Data in 'farm_name' column that is in English: 12.08%


### 7. Region

Actually the most frequent classes only representing ~5% as same as 'mill' and 'farm_name' but I want to explore more from this point. Since I assume the regions can significantly impact quality of coffee so I convert these regions to location represented in [lat, lon].

In [36]:
print(df['region'].value_counts(normalize=True))

region
Chiayi                         0.058537
新竹縣                            0.053659
苗栗縣                            0.034146
North of Thailand              0.034146
Yunlin                         0.034146
                                 ...   
Addis Ababa                    0.004878
Ka'u district of Big Island    0.004878
玉里鎮Yuli Township               0.004878
Ethiopia                       0.004878
Minas Gerais                   0.004878
Name: proportion, Length: 120, dtype: float64


In [37]:
# fill missing values in 'region' with country_of_origin and altitude
non_region_df = df[df['region'].isnull()]
non_region_df[['country_of_origin', 'altitude']].value_counts()

country_of_origin  altitude
Colombia           1625.0      1
Nicaragua          1100.0      1
Name: count, dtype: int64

In [38]:
for index, row in non_region_df.iterrows():
    country = row['country_of_origin']
    altitude = row['altitude']
    if pd.isnull(altitude):
        continue
    region = df[(df['country_of_origin'] == country) & (df['altitude'] == altitude)]['region'].mode().values
    if len(region) > 0:
        print("match: ", region[0])
        df.loc[index, 'region'] = region[0]
    else:
        region = df[(df['country_of_origin'] == country)]['region'].mode().values
        if len(region) > 0:
            print("mode: ", region[0])
            df.loc[index, 'region'] = region[0]

mode:  Quindio
mode:  ESTELI


In [39]:
df[df['region'].isnull()]

,country_of_origin,farm_name,lot_number,mill,ico_number,company,altitude,region,producer,number_of_bags,bag_weight,in_country_partner,harvest_year,grading_date,owner,variety,status,processing_method,aroma,flavor,aftertaste,acidity,body,balance,uniformity,clean_cup,sweetness,overall,defects,total_cup_points,moisture_percentage,category_one_defects,quakers,color,category_two_defects,expiration,certification_body,certification_address,certification_contact


### 8. Lot Number

The same reason to 'mill' and 'farm_name' their most frequent name contain only ~5% of total data. I decided to discard this.

In [40]:
print(df['lot_number'].unique())
df['lot_number'].value_counts(normalize=True)

['CQU2022015' 'The 2022 Pacific Rim Coffee Summit,T037'
 'The 2022 Pacific Rim Coffee Summit,LA01' 'CQU2022017' 'CQU2023002'
 'The 2022 Pacific Rim Coffee Summit,GT02'
 'The 2022 Pacific Rim Coffee Summit,T034'
 'The 2022 Pacific Rim Coffee Summit,T050'
 'The 2022 Pacific Rim Coffee Summit,T018' 'CN 4127230034/4189230113'
 '010/0296/600' 'The 2022 Pacific Rim Coffee Summit,GT12'
 'The 2022 Pacific Rim Coffee Summit,T051'
 'Grade 1, Guji, Natural, Gelana Geisha'
 'The 2022 Pacific Rim Coffee Summit,CO03'
 'The 2022 Pacific Rim Coffee Summit,T014' 'CQU2023006' '202203' '202112'
 'CN 4127230032/P-9140' 'The 2022 Pacific Rim Coffee Summit,GT03'
 '2022/03' '1' 'CQU2023005' 'The 2022 Pacific Rim Coffee Summit, T003'
 'Arara 1' 'The 2022 Pacific Rim Coffee Summit,T024'
 'The 2022 Pacific Rim Coffee Summit,T048'
 'The 2022 Pacific Rim Coffee Summit,T045'
 'The 2022 Pacific Rim Coffee Summit,US01' '2022/3/1(老叢)' '2022/02'
 '2022臺南市精品咖啡評鑑批次' 'Washed Limmu' '0065'
 'The 2022 Pacific Rim Coffee Su

lot_number
1                                          0.053398
202203                                     0.019417
2022臺南市精品咖啡評鑑批次                            0.014563
1-26                                       0.009709
CQU2022018                                 0.009709
                                             ...   
0712                                       0.004854
2021/11                                    0.004854
The 2022 Pacific Rim Coffee Summit,T031    0.004854
11-63-657                                  0.004854
1058 y 1059                                0.004854
Name: proportion, Length: 187, dtype: float64

In [41]:
# plot bar chart for lot_number
fig = go.Figure()
# bar plot
fig.add_trace(go.Bar(x=df['lot_number'].value_counts().index, y=df['lot_number'].value_counts().values, marker_color='blue',))
fig.update_layout(title='Lot Number Distribution', xaxis_title='Lot Number', yaxis_title='Count')
fig.show()

### 9. Producer

The same reason to 'mill', 'farm_name' and 'lot_number' their most frequent name contain only ~3% of total data. I decided to discard this.

In [42]:
print(df['producer'].unique())
df['producer'].value_counts(normalize=True)

['Diego Samuel Bermudez' '曾福森' 'WU TAO CHI' 'Santa Maria de Dota'
 'Camilo Merizalde' 'Emilio Antonio Medina Garcia' '黃保錫' '莊家榮' '鍾旭亮'
 'DORMAN (T) LIMITED' 'Tesfaye Bekele Degaga' 'Karen Acajabon' '賴彥合'
 'Kerchanshe' 'Josue Enciso Suarez' '葉世遠' 'Halo Bariti Cooprative' '陳景立'
 '王瀅惠' 'Fredy Orantes' '游婷淯 YU TING YU' 'Lisu farmers'
 'Farm Bedoya Arango' '蘇甘永' 'Diogo T. Dias de Macedo' '方政倫' '郭志豪' '陳清龍'
 'Kona Hills LLC' '張進義Zhang Jin-Yi' '葉明讚 (YE,MING-ZAN)' 'Eshetu'
 'Bisrat Melaku' 'YHAENU PLC' '許定燁' '陳建源(CHEN, JIAN-YUAN)' 'Small Holder'
 '陳榮豐' '李志祥 Li Zhi-xia' '陳科翰' '黃簡秋蘭' 'Kyagalanyi coffee limited'
 '劉孟宗 LIU MENG TSUNG' '王清連' 'Varios productores' 'PABLO CHUY'
 'san francisco cotzal' '黃俊淦' '戴禎男' 'PT Indo CafCo North Sumatera'
 'Ruya Yayu' 'IYENGA AMCOS' '鄭虞坪' '吳振宏' 'Homero Paiva' 'La Santa' '卓榮華'
 'Varios Productores' 'VALBROS, S. A.' 'Martin Gutierrez' '羅政宏'
 '郭雅聰（KUO,YA-TSUNG）' 'NicaNoLA Coffee' '浦瀚文' '王政淵'
 'TONG PHUC LAM TRADING SERVICES COMPANY LIMITED' '李鴻儀'
 'Jose Nahum Maldona

producer
Doi Tung Development Project                      0.033981
Roselia Yglesias                                  0.014563
Fausto Nahun Maldonado                            0.014563
Martin Gutierrez                                  0.014563
J.J. Borja Nathan, S.A.                           0.014563
                                                    ...   
TONG PHUC LAM TRADING SERVICES COMPANY LIMITED    0.004854
李鴻儀                                               0.004854
TRACON TRADING PLC                                0.004854
黃怡瑄 Huang Yi-xuan                                 0.004854
Walter Matter                                     0.004854
Name: proportion, Length: 172, dtype: float64

In [43]:
# plot bar chart for producer
fig = go.Figure()
# bar plot
fig.add_trace(go.Bar(x=df['producer'].value_counts().index, y=df['producer'].value_counts().values, marker_color='blue',))
fig.update_layout(title='Producer Distribution', xaxis_title='Producer', yaxis_title='Count')
fig.show()

## Data-Typo

In [44]:
# Discard 'mill', 'farm_name', 'lot_number', 'producer' columns
df = df.drop(['mill', 'farm_name', 'lot_number', 'producer'], axis=1)
print(f"Data shape after dropping ['mill', 'farm_name', 'lot_number', 'producer']: {df.shape} | rows: {df.shape[0]} | columns: {df.shape[1]}")

Data shape after dropping ['mill', 'farm_name', 'lot_number', 'producer']: (207, 35) | rows: 207 | columns: 35


In [45]:
numeric_df, object_df = get_info(df)

In [46]:
numeric_df

,column,dtype,valid,missing,unique,unique_pct,mean,std,min,25%,50%,75%,max
0,altitude,float64,207,0,77,37.20,1313.25,720.08,139.00,1020.00,1340.00,1600.000,5400.00
1,number_of_bags,int64,207,0,55,26.57,155.45,244.48,1.00,1.00,14.00,275.000,2240.00
2,aroma,float64,207,0,19,9.18,7.72,0.29,6.50,7.58,7.67,7.920,8.58
3,flavor,float64,207,0,19,9.18,7.74,0.28,6.75,7.58,7.75,7.920,8.50
4,aftertaste,float64,207,0,20,9.66,7.60,0.28,6.67,7.42,7.58,7.750,8.42
5,acidity,float64,207,0,19,9.18,7.69,0.26,6.83,7.50,7.67,7.875,8.58
6,body,float64,207,0,17,8.21,7.64,0.23,6.83,7.50,7.67,7.750,8.25
7,balance,float64,207,0,18,8.70,7.64,0.26,6.67,7.50,7.67,7.790,8.42
8,uniformity,float64,207,0,3,1.45,9.99,0.10,8.67,10.00,10.00,10.000,10.00
9,clean_cup,float64,207,0,1,0.48,10.00,0.00,10.00,10.00,10.00,10.000,10.00


In [47]:
object_df

,column,dtype,valid,missing,unique,unique_pct,top,freq
0,country_of_origin,object,207,0,22,10.63,Taiwan,61
1,ico_number,bool,207,0,2,0.97,False,137
2,company,object,207,0,72,34.78,Taiwan Coffee Laboratory,51
3,region,object,207,0,120,57.97,Chiayi,12
4,bag_weight,object,207,0,39,18.84,30 kg,39
5,in_country_partner,object,207,0,21,10.14,Taiwan Coffee Laboratory 台灣咖啡研究室,83
6,harvest_year,object,207,0,7,3.38,2021 / 2022,99
7,grading_date,object,207,0,75,36.23,"November 15th, 2022",40
8,owner,object,207,0,80,38.65,Taiwan Coffee Laboratory 台灣咖啡研究室,30
9,variety,object,207,0,46,22.22,Gesha,38


In [48]:
# From object_df_new found that 'bag_weight' can convert to numnerical with split kg out
df['bag_weight'] = df['bag_weight'].str.split('kg').str[0].astype(float)
df['bag_weight'].value_counts(normalize=True).iloc[:10]

bag_weight
30.0     0.188406
60.0     0.149758
69.0     0.120773
1.0      0.067633
15.0     0.048309
5.0      0.048309
2.0      0.038647
70.0     0.028986
100.0    0.028986
50.0     0.028986
Name: proportion, dtype: float64

## Exploring-Data

In [49]:
numeric_df, object_df = get_info(df)

### Numerical-Data
- [1. Altitude Outlier](####1.Altitude-Outlier)
- [2. Bag Weight Outlier](####2.Bag-Weight-Outlier)
- [3. Number of Bags Outlier](####3.Number-of-Bags-Outlier)

In [50]:
# correct outliers with IQR method
def correct_outliers(df, column):
    """
    This function corrects outliers using IQR method
    Args:
        df: pd.DataFrame
        column: str
    Returns:
        pd.DataFrame
    """
    q1 = df[column].quantile(0.25)
    q3 = df[column].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    df.loc[df[column] < lower_bound, column] = lower_bound
    df.loc[df[column] > upper_bound, column] = upper_bound
    return df


def plot_box(df: pd.DataFrame, column: str=None):
    """
    This function plots boxplot for the column
    Args:
        df: pd.DataFrame
        column: str
    """
    fig = go.Figure()
    if column:
        fig.add_trace(go.Box(y=df[column], name=column, boxmean=True))
        fig.update_layout(title=f'Distribution of the {column}')
    else:
        numeric_df, _ = get_info(df)
        for i, row in numeric_df.iterrows():
            fig.add_trace(go.Box(y=df[row['column']], name=row['column'], boxmean=True))
        fig.update_layout(title=f'Distribution of the numeric columns')
    fig.show()


In [51]:
# plot boxplot for all numeric columns
plot_box(df)

#### 1.Altitude-Outlier

'altitude' has extreamly outliers, I will check it with region first. From dataset [discussion](https://www.kaggle.com/datasets/fatihb/coffee-quality-data-cqi/discussion/411169) community discuss that 'altitude' is in meters unit but these outliers are in ft unit so I will change it to meters.
- Santa Rosa 
- ANTIGUA GUATEMALA
- Lam Dong Province

In [52]:
# 'altitude' has extreamly outliers, I will check it with region first
df.query("altitude > 3000")

,country_of_origin,ico_number,company,altitude,region,number_of_bags,bag_weight,in_country_partner,harvest_year,grading_date,owner,variety,status,processing_method,aroma,flavor,aftertaste,acidity,body,balance,uniformity,clean_cup,sweetness,overall,defects,total_cup_points,moisture_percentage,category_one_defects,quakers,color,category_two_defects,expiration,certification_body,certification_address,certification_contact
47,Guatemala,True,"OLAM AGRO GUATEMALA, S.A.",4700.0,Santa Rosa,632,30.0,Asociacion Nacional Del Café,2022 / 2023,"February 23rd, 2023",Yesica Alejandra Martìnez Vàsquez,Catuai,Completed,Washed / Wet,7.67,8.00,7.75,7.92,8.00,7.83,10.0,10.0,10.0,7.83,0.0,85.00,11.3,0,0,green,4,"February 23rd, 2024",Asociacion Nacional Del Café,"5a Calle 0-50, Zona 14 Guatemala City, Guatema...",Brayan Cifuentes -
60,Guatemala,True,"VALBROS, S. A.",4895.0,ANTIGUA GUATEMALA,66,30.0,Asociacion Nacional Del Café,2021 / 2022,"May 19th, 2022",Angelica Paola Citan Lopez,Bourbon,Completed,Natural / Dry,7.83,8.00,7.67,7.83,7.75,7.75,10.0,10.0,10.0,7.83,0.0,84.67,10.9,0,3,yellow-green,2,"May 19th, 2023",Asociacion Nacional Del Café,"5a Calle 0-50, Zona 14 Guatemala City, Guatema...",Brayan Cifuentes -
99,Guatemala,True,Asociación Nacional del Cafe,5272.5,ANTIGUA GUATEMALA,635,30.0,Asociacion Nacional Del Café,2021 / 2022,"May 19th, 2022",Angelica Paola Citan Lopez,Bourbon,Completed,Washed / Wet,7.58,7.92,7.50,7.67,7.75,7.67,10.0,10.0,10.0,7.75,0.0,83.83,10.9,0,1,greenish,4,"May 19th, 2023",Asociacion Nacional Del Café,"5a Calle 0-50, Zona 14 Guatemala City, Guatema...",Brayan Cifuentes -
182,Vietnam,False,Brew Baby Coffee Company,5400.0,Lam Dong Province,1,1.0,Firedancer Coffee Consultants,2022 / 2023,"April 10th, 2023",Rodney Murray,Catimor,Completed,Pulped natural / honey,7.50,7.50,7.25,7.42,7.42,7.42,10.0,10.0,10.0,7.33,0.0,81.83,11.2,0,0,green,4,"April 9th, 2024",Firedancer Coffee Consultants,"165 Commerce Drive, Ste 103",Firedancer Coffee Consultants -


In [53]:
# Convert ft to m
df.loc[df['altitude'] > 3000, 'altitude'] = df.loc[df['altitude'] > 3000, 'altitude'] * 0.3048

#### 2.Bag-Weight-Outlier

In [54]:
# 'bag_weight' has extreamly outliers, I will correct it using IQR
df = correct_outliers(df, 'bag_weight')

#### 3.Number-of-Bags-Outlier

In [55]:
# 'number_of_bags' has extreamly outliers, I will correct it using IQR
df = correct_outliers(df, 'number_of_bags')

#### Plot after correct outlier

In [56]:
plot_box(df)

### Categorical-Data

Encoding Categorical Data

- [1. Country of Origin Encoding](####1.Country-of-Origin-Encoding)
- [2. Company Encoding](####2.Company-Encoding)
- [3. Region Encoding](####3.Region-Encoding)
- [4. In Country Partner Encoding](####4.In-Country-Partner-Encoding)
- [5. Harvest Year and Grading Date Encoding](####5.Harvest-Year-and-Grading-Date-Encoding)
- [6. Owner Encoding](####6.Owner-Encoding)
- [7. Variety Encoding](####7.Variety-Encoding)
- [8. Processing Method Encoding](####8.Processing-Method-Encoding)
- [9. Color Encoding](####9.Color-Encoding)
- [10. Certification Body Encoding](####10.Certification-Body-Encoding)
- [11. Certification Address Encoding](####11.Certification-Address-Encoding)
- [12. Certification Contact Encoding](####12.Certification-Contact-Encoding)
- [Assign Categorical Number](####Assign-Categorical-Number)

Note: I discard 'status' since this data is constant for all dataset and 'expiration' since I found that it has constant range to 'grading_date' with 12 months for all dataset.

In [57]:
object_df

,column,dtype,valid,missing,unique,unique_pct,top,freq
0,country_of_origin,object,207,0,22,10.63,Taiwan,61
1,ico_number,bool,207,0,2,0.97,False,137
2,company,object,207,0,72,34.78,Taiwan Coffee Laboratory,51
3,region,object,207,0,120,57.97,Chiayi,12
4,in_country_partner,object,207,0,21,10.14,Taiwan Coffee Laboratory 台灣咖啡研究室,83
5,harvest_year,object,207,0,7,3.38,2021 / 2022,99
6,grading_date,object,207,0,75,36.23,"November 15th, 2022",40
7,owner,object,207,0,80,38.65,Taiwan Coffee Laboratory 台灣咖啡研究室,30
8,variety,object,207,0,46,22.22,Gesha,38
9,status,object,207,0,1,0.48,Completed,207


In [58]:
def group_classes(df, column, threshold=0.01, name='Other'):
    """
    This function groups classes with a threshold
    Args:
        df: pd.DataFrame
        column: str
        threshold: float
    Returns:
        pd.DataFrame
    """
    value_counts = df[column].value_counts(normalize=True)
    classes = value_counts[value_counts < threshold].index
    df.loc[df[column].isin(classes), column] = name
    return df

def group_classes_with_dict(field, groups):
    for group, members in groups.items():
        if field in members:
            return group
    return 'Other'


#### 1.Country-of-Origin-Encoding

In [59]:
# 'country_of_origin'
print(df['country_of_origin'].value_counts())
# rename 'Tanzania, United Republic Of' to 'Tanzania'
df.loc[df['country_of_origin'] == 'Tanzania, United Republic Of', 'country_of_origin'] = 'Tanzania'
# by geographical location
asia_pacific = ['Taiwan', 'Thailand', 'Vietnam', 'Indonesia', 'Laos', 'Myanmar']
central_america = ['Honduras', 'Costa Rica', 'Guatemala', 'Nicaragua', 'Panama', 'El Salvador']
south_america = ['Peru', 'Colombia', 'Brazil']
north_america = ['United States (Hawaii)', 'Mexico']
africa = ['Madagascar', 'Tanzania', 'Uganda', 'Ethiopia', 'Kenya']
df.loc[df['country_of_origin'].isin(asia_pacific), 'country_of_origin'] = 'Asia Pacific'
df.loc[df['country_of_origin'].isin(central_america), 'country_of_origin'] = 'Central America'
df.loc[df['country_of_origin'].isin(south_america), 'country_of_origin'] = 'South America'
df.loc[df['country_of_origin'].isin(north_america), 'country_of_origin'] = 'North America'
df.loc[df['country_of_origin'].isin(africa), 'country_of_origin'] = 'Africa'
print("\nAfter Grouping:")
print(df['country_of_origin'].value_counts())

country_of_origin
Taiwan                          61
Guatemala                       21
Colombia                        19
Honduras                        13
Thailand                        12
Ethiopia                        11
Brazil                          10
Costa Rica                       8
Nicaragua                        7
El Salvador                      7
Tanzania, United Republic Of     6
United States (Hawaii)           5
Mexico                           4
Peru                             4
Vietnam                          4
Uganda                           3
Indonesia                        3
Laos                             3
Panama                           2
Kenya                            2
Madagascar                       1
Myanmar                          1
Name: count, dtype: int64

After Grouping:
country_of_origin
Asia Pacific       84
Central America    58
South America      33
Africa             23
North America       9
Name: count, dtype: int64


In [60]:
df.groupby('country_of_origin')['total_cup_points'].mean().sort_values(ascending=False)

country_of_origin
Africa             84.626087
Asia Pacific       83.997500
Central America    83.347414
North America      83.232222
South America      83.085758
Name: total_cup_points, dtype: float64

In [61]:
df.groupby('country_of_origin')['total_cup_points'].mean().sort_values(ascending=False)

country_of_origin
Africa             84.626087
Asia Pacific       83.997500
Central America    83.347414
North America      83.232222
South America      83.085758
Name: total_cup_points, dtype: float64

In [62]:
# average total_cup_points by country_of_origin
df_tmp = df.groupby('country_of_origin')['total_cup_points'].mean().sort_values(ascending=False)
# plot bar chart for country_of_origin
fig = go.Figure()
# bar plot
fig.add_trace(go.Bar(x=df_tmp.index, y=df_tmp.values,))
fig.update_layout(title='Country of Origin Distribution', xaxis_title='Total Cup Points of Origin', yaxis_title='Count')
fig.show()

#### 2.Company-Encoding

In [63]:
# 'company'
print(df['company'].value_counts())
# group classes with a threshold
df_temp = group_classes(df.copy(), 'company', threshold=0.01)
print("\nAfter Grouping:")
print(df_temp['company'].value_counts())

company
Taiwan Coffee Laboratory        51
Taiwu Coffee Cooperative        25
Coffee Quality Union            15
Doi Tung Development Project     7
Peter Schoenfeld, S.A.           6
                                ..
Taylor Winch Coffee Ltd          1
ECOM COLOMBIA                    1
Exportadora Café California      1
Coffee Quality Institute         1
marubeni                         1
Name: count, Length: 72, dtype: int64

After Grouping:
company
Other                                70
Taiwan Coffee Laboratory             51
Taiwu Coffee Cooperative             25
Coffee Quality Union                 15
Doi Tung Development Project          7
Peter Schoenfeld, S.A.                6
Mercon Honduras                       4
Marubeni Corporation                  4
Mr.Brown Cafe                         4
宸嶧國際有限公司                              3
InterAmerican Coffee                  3
CECA S.A.                             3
Cafe Organico Marcala S.A de C.V.     3
Consejo Salvadoreño del 

After grouping 'company' which has their count < 1% to 'Other' class, This class becomes majority class in this data so I will leave it for now.

#### 3.Region-Encoding

Actually the most frequent classes only representing ~5% but I want to explore more from this point. Since I assume the regions can significantly impact quality of coffee so I convert these regions to location represented in [lat, lon].

In [64]:
# region
print(df['region'].value_counts(normalize=True))

region
Chiayi                         0.057971
新竹縣                            0.053140
苗栗縣                            0.033816
North of Thailand              0.033816
Yunlin                         0.033816
                                 ...   
Addis Ababa                    0.004831
Ka'u district of Big Island    0.004831
玉里鎮Yuli Township               0.004831
Ethiopia                       0.004831
Minas Gerais                   0.004831
Name: proportion, Length: 120, dtype: float64


In [65]:
# read the coordinates from the json file
with open('data/region_lat_lon.json', 'r') as f:
    region_lat_lon = json.load(f)
# print the first 5 items
print(list(region_lat_lon.items())[:5])

[('Piendamo,Cauca', [2.636706, -76.5061489]), ('Chiayi', [23.4518428, 120.2554615]), ('Laos Borofen Plateau', [15.0, 106.0]), ('Los Santos,Tarrazu', [9.65965, -84.02138]), ('Popayan,Cauca', [2.45417, -76.60917])]


In [66]:
lat_lon_df = df.copy()
# map the region to the coordinates
lat_lon_df['region_lat_lon'] = lat_lon_df['region'].map(region_lat_lon)
# split the coordinates into latitude and longitude
lat_lon_df[['region_lat', 'region_lon']] = pd.DataFrame(lat_lon_df['region_lat_lon'].tolist(), index=lat_lon_df.index)
# check the missing values
lat_lon_df[lat_lon_df['region_lat_lon'].isnull()]
# add to df
df['region_lat'] = lat_lon_df['region_lat']
df['region_lon'] = lat_lon_df['region_lon']

In [67]:
# plot the location of region_lat_lon on the map on plotly and show the total_cup_points
# coffee_belt_lat = [25, -30]
# fig = go.Figure()

# fig.add_trace(go.Scattergeo(
#     lat=df['region_lat'],
#     lon=df['region_lon'],
#     text=df['region'],
#     marker=dict(
#         size=df['total_cup_points']/10,
#         color=df['total_cup_points'],
#         colorscale='Viridis',
#         colorbar=dict(title='Total Cup Points'),
#         line=dict(width=0.5, color='rgba(40,40,40,0.5)')
#     ),
#     mode='markers',
#     hoverinfo='text'
# ))

# fig.update_layout(
#     title='Coffee Quality by Region',
#     geo=dict(
#         projection_type='natural earth'
#     )
# )
fig = px.scatter_geo(df, 
                     lat='region_lat', 
                     lon='region_lon', 
                     color='total_cup_points', 
                     hover_name='region', 
                     size='total_cup_points',
                     projection='natural earth')
fig.update_geos(showcountries=True, countrycolor="Black", showland=True, showocean=True, oceancolor="LightBlue")
fig.show()

In [68]:
# average total_cup_points by region that in coffee_belt and not in coffee_belt
coffee_belt_lat = [25, -30]
df_tmp = df.copy()
df_tmp['coffee_belt'] = df_tmp.apply(lambda x: 1 if (x['region_lat'] > coffee_belt_lat[1]) and (x['region_lat'] < coffee_belt_lat[0]) else 0, axis=1)

In [69]:
# average total_cup_points by coffee_belt
df_tmp = df_tmp.groupby('coffee_belt')['total_cup_points'].mean().sort_values(ascending=False)
# plot bar chart for country_of_origin
fig = go.Figure()
# bar plot
fig.add_trace(go.Bar(x=df_tmp.index, y=df_tmp.values,))
fig.update_layout(title='Coffee Belt Distribution', xaxis_title='Coffee Belt', yaxis_title='Total Cup Points')
fig.show()

#### 4.In-Country-Partner-Encoding

In [70]:
print(df['in_country_partner'].value_counts(normalize=True))

a = group_classes(df.copy(), 'in_country_partner', threshold=0.01)
a['in_country_partner'].value_counts(normalize=True)

in_country_partner
Taiwan Coffee Laboratory 台灣咖啡研究室                                     0.400966
Japan Coffee Exchange                                                0.130435
Asociacion Nacional Del Café                                         0.067633
Instituto Hondureño del Café                                         0.048309
FABB Academy of Coffee                                               0.043478
Kenya Coffee Traders Association                                     0.038647
Specialty Coffee Association of Costa Rica                           0.033816
Blossom Valley International宸嶧國際                                     0.028986
METAD Agricultural Development plc                                   0.028986
Salvadoran Coffee Council                                            0.028986
Centro Agroecológico del Café A.C.                                   0.024155
Brazil Specialty Coffee Association                                  0.019324
NKG Quality Service (a division of Bernhard R

in_country_partner
Taiwan Coffee Laboratory 台灣咖啡研究室                                     0.400966
Japan Coffee Exchange                                                0.130435
Asociacion Nacional Del Café                                         0.067633
Instituto Hondureño del Café                                         0.048309
FABB Academy of Coffee                                               0.043478
Kenya Coffee Traders Association                                     0.038647
Specialty Coffee Association of Costa Rica                           0.033816
Salvadoran Coffee Council                                            0.028986
Blossom Valley International宸嶧國際                                     0.028986
METAD Agricultural Development plc                                   0.028986
Centro Agroecológico del Café A.C.                                   0.024155
Brazil Specialty Coffee Association                                  0.019324
Specialty Coffee Association                 

In [71]:
# average total_cup_points by in_country_partner
df_tmp = df.groupby('in_country_partner')['total_cup_points'].mean().sort_values(ascending=False)
# plot bar chart for country_of_origin
fig = go.Figure()
# bar plot
fig.add_trace(go.Bar(x=df_tmp.index, y=df_tmp.values,))
fig.update_layout(title='In Country Partner Distribution', xaxis_title='In Country Partner', yaxis_title='Total Cup Points')
fig.show()

#### 5.Harvest-Year-and-Grading-Date-Encoding

In [72]:
def calculate_harvest_grading_period(harvest_year, grading_date):
    """
    This function calculates the harvest grading period
    Args:
        harvest_year: int
        grading_date: str
    Returns:
        harvest_grading_period: int
    """
    span = re.match(r"(\d{4}) / (\d{4})", harvest_year)
    if span:
        harvest_year = int(span.group(1))
    harvest_date = pd.to_datetime(f"{harvest_year}-01-01")
    grading_year = pd.to_datetime(grading_date)
    period_days = (grading_year - harvest_date).days
    period_months = period_days // 30
    return period_months

In [73]:
# calculate harvest grading period
df['harvest_grading_period'] = df.apply(lambda x: calculate_harvest_grading_period(x['harvest_year'], x['grading_date']), axis=1)
print(df['harvest_grading_period'].value_counts()[:10])

harvest_grading_period
22    38
24    32
15    22
16    13
10    12
6     11
13     9
17     9
14     7
12     6
Name: count, dtype: int64


#### 6.Owner-Encoding

In [74]:
# print(df['owner'].unique())
print(df['owner'].value_counts(normalize=True))
# group classes with a threshold
df_temp = group_classes(df.copy(), 'owner', threshold=0.01)
print("\nAfter Grouping:")
print(df_temp['owner'].value_counts(normalize=True))

owner
Taiwan Coffee Laboratory 台灣咖啡研究室     0.144928
Taiwu                                0.120773
Coffee Quality Union                 0.072464
Yesica Alejandra Martìnez Vàsquez    0.038647
Doi Tung Development Project         0.033816
                                       ...   
Sucafina Colombia SAS                0.004831
Marc Dumont                          0.004831
Kao,SIng-jay                         0.004831
Taylor Winch (Coffee) Ltd.           0.004831
SAJONIA ESTATE COFFEE S.A.           0.004831
Name: proportion, Length: 80, dtype: float64

After Grouping:
owner
Other                                                     0.352657
Taiwan Coffee Laboratory 台灣咖啡研究室                          0.144928
Taiwu                                                     0.120773
Coffee Quality Union                                      0.072464
Yesica Alejandra Martìnez Vàsquez                         0.038647
Doi Tung Development Project                              0.033816
Angelica Paola Cit

After grouping 'owner' which has their count < 1% to 'Other' class, This class becomes majority class in this data so I will leave it for now.

#### Variety-Encoding

Reduce the coffee variety to 12 meaningful groups based on:
- Genetic relationships (Bourbon, Typica, Caturra families)
- Disease resistance characteristics (Catimor Group)
- Regional significance (Ethiopian, Indonesian varieties)
- Specialty status (Gesha group)

In [75]:
variety_groups = {
    'Bourbon Family': [
        'Bourbon', 'Yellow Bourbon', 'Red Bourbon', 'Bourbon Sidra', 
        'Red Bourbon,Caturra'  # Hybrids with Bourbon go here
    ],
    
    'Caturra Family': [
        'Caturra', 'Caturra-Catuai', 'Caturra,Colombia,Castillo',
        'Castillo,Caturra,Bourbon'
    ],
    
    'Catuai Family': [
        'Catuai', 'Yellow Catuai', 'Catuai and Mundo Novo'
    ],
    
    'Typica Family': [
        'Typica', 'Typica + SL34', 'Typica Gesha', 'Maragogype'  # Maragogype is a Typica mutation
    ],
    
    'Gesha/Specialty': [
        'Gesha', 'Typica Gesha', 'Sl34+Gesha', 'Pacamara'
    ],
    
    'SL Varieties': [
        'SL28', 'SL34', 'SL14', 'SL28,SL34,Ruiru11'
    ],
    
    'Catimor Group': [
        'Catimor', 'Sarchimor', 'Catucai', 'Parainema', 'Lempira'  # Rust-resistant varieties
    ],
    
    'Castillo Group': [
        'Castillo', 'Castillo Paraguaycito', 'Castillo and Colombia blend'
    ],
    
    'Regional Varieties': [
        'Ethiopian Heirlooms', 'Java', 'Gayo', 'Wolishalo,Kurume,Dega',
        'Jember,TIM-TIM,Ateng', 'Santander', 'Catrenic'
    ],
    
    'Mundo Novo Group': [
        'Mundo Novo', 'Catuai and Mundo Novo'
    ],
    
    'Mixed Varieties': [
        'BOURBON, CATURRA Y CATIMOR', 'Typica Bourbon Caturra Catimor',
        'Bourbon, Catimor, Caturra, Typica', 'Catimor,Catuai,Caturra,Bourbon',
        'MARSELLESA, CATUAI, CATURRA & MARSELLESA, ANACAFE 14, CATUAI'
    ],
    
    'Other': [
        'SHG', 'Pacas'  # SHG is actually a grade, not a variety
    ]
}


In [76]:
# variety_group
df['variety'] = df['variety'].apply(lambda x: group_classes_with_dict(x, variety_groups))
df['variety'].value_counts()

variety
Gesha/Specialty       41
Caturra Family        33
Typica Family         29
Bourbon Family        23
Catimor Group         18
Regional Varieties    17
Catuai Family         15
SL Varieties          14
Mundo Novo Group       5
Mixed Varieties        5
Other                  4
Castillo Group         3
Name: count, dtype: int64

In [77]:
# average total_cup_points by variety
df_tmp = df.groupby('variety')['total_cup_points'].mean().sort_values(ascending=False)
# plot bar chart for variety
fig = go.Figure()
# bar plot
fig.add_trace(go.Bar(x=df_tmp.index, y=df_tmp.values,))
fig.update_layout(title='Variety Distribution', xaxis_title='Variety', yaxis_title='Total Cup Points')
fig.show()

#### 8.Processing-Method-Encoding

Reduce the processing method to 5 main groups based on:
[Link](https://www.wearecoffeeco.com/blogs/news/coffee-processing-methods?srsltid=AfmBOoqP2CVT_vR_Q9BCDLjoQehyHzv6QvV7DFF8jUiVa8CVSvk6PUXs)


In [78]:
# Define processing method groups
processing_groups = {
    'Washed/Wet Process': [
        'Washed / Wet', 
        'SEMI-LAVADO'  # Spanish for semi-washed
    ],
    
    'Natural/Dry Process': [
        'Natural / Dry'
    ],
    
    'Honey/Pulped Natural Process': [
        'Pulped natural / honey',
        'Honey,Mossto',
    ],
    
    'Wet Hulling Process': [
        'Wet Hulling',  # Indonesian method
        'Semi Washed'
    ],
    
    'Experimental/Anaerobic Process': [
        'Double Anaerobic Washed',
        'Double Carbonic Maceration / Natural',
        'Anaerobico 1000h'
    ]
}

In [79]:
# processing_method
df['processing_method'] = df['processing_method'].apply(lambda x: group_classes_with_dict(x, processing_groups))
print(df['processing_method'].value_counts(normalize=True))
# group classes with a threshold
df = group_classes(df.copy(), 'processing_method', threshold=0.05)
print("\nAfter Grouping:")
print(df['processing_method'].value_counts(normalize=True))

processing_method
Washed/Wet Process                0.618357
Natural/Dry Process               0.222222
Honey/Pulped Natural Process      0.135266
Experimental/Anaerobic Process    0.014493
Wet Hulling Process               0.009662
Name: proportion, dtype: float64

After Grouping:
processing_method
Washed/Wet Process              0.618357
Natural/Dry Process             0.222222
Honey/Pulped Natural Process    0.135266
Other                           0.024155
Name: proportion, dtype: float64


In [80]:
# average total_cup_points by processing_method
df_tmp = df.groupby('processing_method')['total_cup_points'].mean().sort_values(ascending=False)
# plot bar chart for processing_method
fig = go.Figure()
# bar plot
fig.add_trace(go.Bar(x=df_tmp.index, y=df_tmp.values,))
fig.update_layout(title='Processing Method Distribution', xaxis_title='Processing Method', yaxis_title='Total Cup Points')
fig.show()

#### 9.Color-Encoding

In [81]:
# Dictionary for color grouping
color_groups = {
    'Green': ['green', 'greenish'],
    'Blue-Green': ['blue-green', 'bluish-green'],
    'Yellow-Green': ['yellow-green', 'yellow green', 'yellow- green', 'yellow-green', 'yello-green'],
    'Yellow': ['yellowish', 'pale yellow'],
    'Brown/Brownish-Green': ['brownish', 'brownish-green', 'brownish-green', 'browish-green'],
    'Other': []  # Default category for any colors not matched above
}

In [82]:
# color
df['color'] = df['color'].apply(lambda x: group_classes_with_dict(x, color_groups))
print(df['color'].value_counts(normalize=True))

color
Green                   0.661836
Blue-Green              0.159420
Yellow-Green            0.082126
Yellow                  0.048309
Brown/Brownish-Green    0.048309
Name: proportion, dtype: float64


In [83]:
# average total_cup_points by color
df_tmp = df.groupby('color')['total_cup_points'].mean().sort_values(ascending=False)
# plot bar chart for color
fig = go.Figure()
# bar plot
fig.add_trace(go.Bar(x=df_tmp.index, y=df_tmp.values,))
fig.update_layout(title='Color Distribution', xaxis_title='Color', yaxis_title='Total Cup Points')
fig.show()

#### 10.Certification-Body-Encoding

In [84]:
print(df['certification_body'].unique())
df['certification_body'].value_counts(normalize=True)
# group classes with a threshold
df = group_classes(df.copy(), 'certification_body', threshold=0.01)
print("\nAfter Grouping:")
print(df['certification_body'].value_counts(normalize=True))

['Japan Coffee Exchange' 'Taiwan Coffee Laboratory 台灣咖啡研究室'
 'Kenya Coffee Traders Association' 'METAD Agricultural Development plc'
 'Blossom Valley International宸嶧國際' 'FABB Academy of Coffee'
 'Brazil Specialty Coffee Association'
 'Uganda Coffee Development Authority' 'Asociacion Nacional Del Café'
 'NKG Quality Service (a division of Bernhard Rothfos Intercafé AG)'
 'Specialty Coffee Association of Costa Rica'
 'Specialty Coffee Association' 'Instituto Hondureño del Café'
 'ASOCIACIÓN COLOMBIANA PARA LA EXCELENCIA DEL CAFE'
 'Centro Agroecológico del Café A.C.' 'Barista and Coffee Academy of Asia'
 'Specialty Coffee Association of Indonesia'
 'Asociación de Cafés Especiales de Nicaragua'
 'Specialty Coffee Institute of Thailand' 'Salvadoran Coffee Council'
 'Firedancer Coffee Consultants']

After Grouping:
certification_body
Taiwan Coffee Laboratory 台灣咖啡研究室                                     0.400966
Japan Coffee Exchange                                                0.130435
Aso

In [85]:
# average total_cup_points by certification_body
df_tmp = df.groupby('certification_body')['total_cup_points'].mean().sort_values(ascending=False)
# plot bar chart for certification_body
fig = go.Figure()
# bar plot
fig.add_trace(go.Bar(x=df_tmp.index, y=df_tmp.values,))
fig.update_layout(title='Certification Body Distribution', xaxis_title='Certification Body', yaxis_title='Total Cup Points')
fig.show()

#### 11.Certification-Address-Encoding

In [86]:
# print(df['certification_address'].unique())
df['certification_address'].value_counts(normalize=True)
# # group classes with a threshold
df = group_classes(df.copy(), 'certification_address', threshold=0.01)
print("\nAfter Grouping:")
print(df['certification_address'].value_counts(normalize=True))


After Grouping:
certification_address
QAHWAH CO., LTD 4F, No. 225, Sec. 3, Beixin Rd., Xindian Dist. New Taipei City, Taiwan                                                                   0.400966
〒413-0002 静岡県熱海市伊豆山１１７３−５８ 1173-58 Izusan, Atami, Shizuoka, 413-0002 JAPAN                                                                               0.130435
5a Calle 0-50, Zona 14 Guatemala City, Guatemala 1014                                                                                                    0.067633
Laboratorio de Control de Calidad de Café ( CNCC), 33 Calle, 1-2 Avenida, Sector El Cacao. San Pedro Sula, Cortes                                        0.048309
FABB ACADEMY OF COFFEE 259/313 SUKHUMVIT 71 RD., SOI PRIDI BANOMYONG 15, VHADANA BKK, 10110 THAILAND                                                     0.043478
P.O. Box 646 00100Nairobi, Kenya, Denniss Pritt Road, Opposite St Georges Girls Secondary School                                                       

#### 12.Certification-Contact-Encoding

In [87]:
# print(df['certification_contact'].unique())
df['certification_contact'].value_counts(normalize=True)
# # group classes with a threshold
df = group_classes(df.copy(), 'certification_contact', threshold=0.01)
print("\nAfter Grouping:")
print(df['certification_contact'].value_counts(normalize=True))


After Grouping:
certification_contact
Lin, Jen-An Neil 林仁安 - 886-289116612                             0.400966
松澤　宏樹　Koju Matsuzawa - +81(0)9085642901                          0.130435
Brayan Cifuentes -                                               0.067633
Ramon Reyes - + 504 9693-9042                                    0.048309
Janejira Kamonsawedkun - +(66) 095 095 9593                      0.043478
Benson Kibicho Kamau - +254 786 821 621                          0.038647
Noelia Villalobos - (506) 2220 0685                              0.033816
Tomas Bonilla - (503) 2505-6600                                  0.028986
Damon Chen陳嘉峻 - +886423022323                                    0.028986
Aman Adinew (Emebet Dinku) - +251-116-292534, +251-911-519196    0.028986
Stephany Escamilla Femat - Tel. +52 228 8421800 Ext. 3092        0.024155
Chris Allen - 55 35 3212-4705                                    0.019324
Brit Amell (CQI) - bamell@coffeeinstitute.org -                  0.019324

#### Assign-Categorical-Number

In [88]:
# map to number
df['country_of_origin_code'] = df['country_of_origin'].astype('category').cat.codes
df['company_code'] = df['company'].astype('category').cat.codes
df['in_country_partner_code'] = df['in_country_partner'].astype('category').cat.codes
df['harvest_grading_period_code'] = df['harvest_grading_period'].astype('category').cat.codes
df['owner_code'] = df['owner'].astype('category').cat.codes
df['variety_code'] = df['variety'].astype('category').cat.codes
df['processing_method_code'] = df['processing_method'].astype('category').cat.codes
df['color_code'] = df['color'].astype('category').cat.codes
df['certification_body_code'] = df['certification_body'].astype('category').cat.codes
df['certification_address_code'] = df['certification_address'].astype('category').cat.codes
df['certification_contact_code'] = df['certification_contact'].astype('category').cat.codes


In [89]:
df.head()

,country_of_origin,ico_number,company,altitude,region,number_of_bags,bag_weight,in_country_partner,harvest_year,grading_date,owner,variety,status,processing_method,aroma,flavor,aftertaste,acidity,body,balance,uniformity,clean_cup,sweetness,overall,defects,total_cup_points,moisture_percentage,category_one_defects,quakers,color,category_two_defects,expiration,certification_body,certification_address,certification_contact,region_lat,region_lon,harvest_grading_period,country_of_origin_code,company_code,in_country_partner_code,harvest_grading_period_code,owner_code,variety_code,processing_method_code,color_code,certification_body_code,certification_address_code,certification_contact_code
0,South America,False,Coffee Quality Union,1815.0,"Piendamo,Cauca",1,35.0,Japan Coffee Exchange,2021 / 2022,"September 21st, 2022",Coffee Quality Union,Castillo Group,Completed,Other,8.58,8.50,8.42,8.58,8.25,8.42,10.0,10.0,10.0,8.58,0.0,89.33,11.8,0,0,Green,3,"September 21st, 2023",Japan Coffee Exchange,"〒413-0002 静岡県熱海市伊豆山１１７３−５８ 1173-58 Izusan, Ata...",松澤 宏樹 Koju Matsuzawa - +81(0)9085642901,2.636706,-76.506149,20,4,12,10,16,13,1,2,2,8,17,17
1,Asia Pacific,False,Taiwan Coffee Laboratory,1200.0,Chiayi,1,80.0,Taiwan Coffee Laboratory 台灣咖啡研究室,2021 / 2022,"November 15th, 2022",Taiwan Coffee Laboratory 台灣咖啡研究室,Gesha/Specialty,Completed,Washed/Wet Process,8.50,8.50,7.92,8.00,7.92,8.25,10.0,10.0,10.0,8.50,0.0,87.58,10.5,0,0,Blue-Green,0,"November 15th, 2023",Taiwan Coffee Laboratory 台灣咖啡研究室,"QAHWAH CO., LTD 4F, No. 225, Sec. 3, Beixin Rd...","Lin, Jen-An Neil 林仁安 - 886-289116612",23.451843,120.255461,22,1,57,19,18,59,5,3,0,16,15,9
2,Asia Pacific,False,Taiwan Coffee Laboratory,1300.0,Laos Borofen Plateau,19,25.0,Taiwan Coffee Laboratory 台灣咖啡研究室,2021 / 2022,"November 15th, 2022",Taiwan Coffee Laboratory 台灣咖啡研究室,Regional Varieties,Completed,Other,8.33,8.42,8.08,8.17,7.92,8.17,10.0,10.0,10.0,8.33,0.0,87.42,10.4,0,0,Yellow,2,"November 15th, 2023",Taiwan Coffee Laboratory 台灣咖啡研究室,"QAHWAH CO., LTD 4F, No. 225, Sec. 3, Beixin Rd...","Lin, Jen-An Neil 林仁安 - 886-289116612",15.000000,106.000000,22,1,57,19,18,59,9,2,3,16,15,9
3,Central America,False,Coffee Quality Union,1900.0,"Los Santos,Tarrazu",1,22.0,Japan Coffee Exchange,2022,"September 21st, 2022",Coffee Quality Union,Gesha/Specialty,Completed,Washed/Wet Process,8.08,8.17,8.17,8.25,8.17,8.08,10.0,10.0,10.0,8.25,0.0,87.17,11.8,0,0,Green,0,"September 21st, 2023",Japan Coffee Exchange,"〒413-0002 静岡県熱海市伊豆山１１７３−５８ 1173-58 Izusan, Ata...",松澤 宏樹 Koju Matsuzawa - +81(0)9085642901,9.659650,-84.021380,8,2,12,10,4,13,5,3,2,8,17,17
4,South America,False,Coffee Quality Union,1975.0,"Popayan,Cauca",2,24.0,Japan Coffee Exchange,2022,"March 6th, 2023",Coffee Quality Union,Bourbon Family,Completed,Honey/Pulped Natural Process,8.33,8.33,8.08,8.25,7.92,7.92,10.0,10.0,10.0,8.25,0.0,87.08,11.6,0,2,Yellow-Green,2,"March 5th, 2024",Japan Coffee Exchange,"〒413-0002 静岡県熱海市伊豆山１１７３−５８ 1173-58 Izusan, Ata...",松澤 宏樹 Koju Matsuzawa - +81(0)9085642901,2.454170,-76.609170,14,4,12,10,10,13,0,0,4,8,17,17


## Remove-Irrevant-Data

In [90]:
features_nummeric, _ = get_info(df)
features_nummeric

,column,dtype,valid,missing,unique,unique_pct,mean,std,min,25%,50%,75%,max
0,altitude,float64,207,0,77,37.20,1245.18,489.81,139.000000,1020.000000,1340.000000,1600.000000,2361.000000
1,number_of_bags,int64,207,0,54,26.09,146.62,195.43,1.000000,1.000000,14.000000,275.000000,686.000000
2,bag_weight,float64,207,0,36,17.39,41.39,30.89,1.000000,15.000000,30.000000,60.000000,127.500000
3,aroma,float64,207,0,19,9.18,7.72,0.29,6.500000,7.580000,7.670000,7.920000,8.580000
4,flavor,float64,207,0,19,9.18,7.74,0.28,6.750000,7.580000,7.750000,7.920000,8.500000
5,aftertaste,float64,207,0,20,9.66,7.60,0.28,6.670000,7.420000,7.580000,7.750000,8.420000
6,acidity,float64,207,0,19,9.18,7.69,0.26,6.830000,7.500000,7.670000,7.875000,8.580000
7,body,float64,207,0,17,8.21,7.64,0.23,6.830000,7.500000,7.670000,7.750000,8.250000
8,balance,float64,207,0,18,8.70,7.64,0.26,6.670000,7.500000,7.670000,7.790000,8.420000
9,uniformity,float64,207,0,3,1.45,9.99,0.10,8.670000,10.000000,10.000000,10.000000,10.000000


In [99]:
list(features_nummeric['column'].values)

['altitude',
 'number_of_bags',
 'bag_weight',
 'aroma',
 'flavor',
 'aftertaste',
 'acidity',
 'body',
 'balance',
 'uniformity',
 'clean_cup',
 'sweetness',
 'overall',
 'defects',
 'total_cup_points',
 'moisture_percentage',
 'category_one_defects',
 'quakers',
 'category_two_defects',
 'region_lat',
 'region_lon',
 'harvest_grading_period',
 'country_of_origin_code',
 'company_code',
 'in_country_partner_code',
 'harvest_grading_period_code',
 'owner_code',
 'variety_code',
 'processing_method_code',
 'color_code',
 'certification_body_code',
 'certification_address_code',
 'certification_contact_code']

In [104]:
features_list = list(features_nummeric['column'].values)
# pop constant 'clean_cup', 'sweetness', 'defects'
features_list.remove('clean_cup')
features_list.remove('sweetness')
features_list.remove('defects')
features_df = df[features_list]
features_df.head()

,altitude,number_of_bags,bag_weight,aroma,flavor,aftertaste,acidity,body,balance,uniformity,overall,total_cup_points,moisture_percentage,category_one_defects,quakers,category_two_defects,region_lat,region_lon,harvest_grading_period,country_of_origin_code,company_code,in_country_partner_code,harvest_grading_period_code,owner_code,variety_code,processing_method_code,color_code,certification_body_code,certification_address_code,certification_contact_code
0,1815.0,1,35.0,8.58,8.50,8.42,8.58,8.25,8.42,10.0,8.58,89.33,11.8,0,0,3,2.636706,-76.506149,20,4,12,10,16,13,1,2,2,8,17,17
1,1200.0,1,80.0,8.50,8.50,7.92,8.00,7.92,8.25,10.0,8.50,87.58,10.5,0,0,0,23.451843,120.255461,22,1,57,19,18,59,5,3,0,16,15,9
2,1300.0,19,25.0,8.33,8.42,8.08,8.17,7.92,8.17,10.0,8.33,87.42,10.4,0,0,2,15.000000,106.000000,22,1,57,19,18,59,9,2,3,16,15,9
3,1900.0,1,22.0,8.08,8.17,8.17,8.25,8.17,8.08,10.0,8.25,87.17,11.8,0,0,0,9.659650,-84.021380,8,2,12,10,4,13,5,3,2,8,17,17
4,1975.0,2,24.0,8.33,8.33,8.08,8.25,7.92,7.92,10.0,8.25,87.08,11.6,0,2,2,2.454170,-76.609170,14,4,12,10,10,13,0,0,4,8,17,17


In [121]:
# correlation matrix
correlation = features_df.corr()
fig = go.Figure(data=go.Heatmap(
        z=correlation,
        x=correlation.columns,
        y=correlation.columns,
        colorscale='Blackbody',
        text=correlation.values,  # The values to display
        texttemplate="%{text:.2f}",  # Format to 2 decimal places
        textfont={"size":5},
        hovertemplate='Correlation: %{text:.3f}<extra></extra>'))
fig.update_layout(title='Correlation Matrix', width=800, height=800)
fig.show()

In [122]:
# correlation matrix with annotation filter x only 'total_cup_points'
correlation = features_df.corr()
fig = go.Figure(data=go.Heatmap(
        z=correlation[['total_cup_points']],
        x=['total_cup_points'],
        y=correlation.columns,
        colorscale='Blackbody',
        text=correlation[['total_cup_points']].values,  # The values to display
        texttemplate="%{text:.2f}",  # Format to 2 decimal places
        textfont={"size":10},
        hovertemplate='Correlation: %{text:.3f}<extra></extra>'),
        )
fig.update_layout(title='Correlation Matrix with Total Cup Points', height=800)